In [48]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path
# import logging
# log = logging.getLogger()
# log.setLevel(logging.DEBUG)

from io_utils import *
from peh_utils import * 
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
#### MULTI SESSION

animal_id = 'R600'
bdata = dj.create_virtual_module("new_acquisition", "bdatatest")
subject_session_key = {"ratname": animal_id}

pd_blobs = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids, dates, trials = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

peh_blobs = (bdata.ParsedEvents * bdata.Sessions & subject_session_key).fetch('peh', as_dict=True)

trial_filter = (trials != 0) & (trials != 1)
pd_blobs, peh_blobs, trials, sess_ids, dates = drop_sessions(
    pd_blobs, peh_blobs, trials, sess_ids, dates, trial_filter
)


pd_dicts = convert_to_dict(pd_blobs)
reformat_pd_dict(pd_dicts)

peh_dicts = convert_to_dict(peh_blobs)

assert len(pd_dicts) == len(peh_dicts) == len(trials) == len(sess_ids) == len(dates) , (
    "number of sessions does not match, assumptions of code below are broken!"
)

peh_dicts, peh_dicts, trials, sess_ids, dates = find_and_fix_len_errors(
    pd_dicts, peh_dicts, trials, sess_ids, dates
)


peh_dicts_for_df = get_peh_vars_for_df(peh_dicts, trials, sess_ids)

df = generate_trials_df(animal_id, pd_dicts, peh_dicts_for_df, trials, sess_ids, dates)



dropping 1 sessions of 93
dropping 0 sessions of 92


In [3]:

# #### SINGLE SESSION ####
# session_key = {'sessid': 884257}

# prb = (bdata.Sessions & session_key).fetch(
#     "protocol_data", as_dict=True
# )

# s_id, d, t = (bdata.Sessions & session_key).fetch(
#     "sessid", "sessiondate", "n_done_trials"
# )
# prd = convert_to_dict(prb)
# pd_prepare_dict_for_df(prd)
# session_key = {'sessid': s_id[0]}
# pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
# pehd = bt.transform_blob(pehb[0]['peh'])

